In [24]:
import or_gym
from or_gym import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ray
from ray import tune
from ray.rllib import agents
import os
import pickle

In [17]:
env_name = 'InvManagement-v2'
ray_path = '/home/christian/ray_results/PPO'
iv2_paths = [p for p in os.listdir(ray_path) if env_name in p]
iv2_paths

['PPO_InvManagement-v2_ea14d_00001_1_lr=0.0001,vf_loss_coeff=1.0_2020-11-04_18-01-36',
 'PPO_InvManagement-v2_5c8d9_00008_8_lr=1e-06,vf_loss_coeff=0.1_2020-11-04_12-42-49',
 'PPO_InvManagement-v2_5c8d9_00002_2_lr=1e-06,vf_loss_coeff=1.0_2020-11-04_12-38-45',
 'PPO_InvManagement-v2_ea14d_00003_3_lr=1e-05,vf_loss_coeff=0.5_2020-11-05_04-18-55',
 'PPO_InvManagement-v2_5c8d9_00000_0_lr=1e-05,vf_loss_coeff=1.0_2020-11-04_12-37-31',
 'PPO_InvManagement-v2_ea14d_00005_5_lr=1e-06,vf_loss_coeff=0.5_2020-11-05_09-10-11',
 'PPO_InvManagement-v2_ea14d_00000_0_lr=1e-05,vf_loss_coeff=1.0_2020-11-04_12-55-48',
 'PPO_InvManagement-v2_ea14d_00002_2_lr=1e-06,vf_loss_coeff=1.0_2020-11-04_23-12-13',
 'PPO_InvManagement-v2_5c8d9_00006_6_lr=1e-05,vf_loss_coeff=0.1_2020-11-04_12-41-27',
 'PPO_InvManagement-v2_5c8d9_00004_4_lr=0.0001,vf_loss_coeff=0.5_2020-11-04_12-40-05',
 'PPO_InvManagement-v2_5c8d9_00007_7_lr=0.0001,vf_loss_coeff=0.1_2020-11-04_12-42-09',
 'PPO_InvManagement-v2_5c8d9_00001_1_lr=0.0001,vf_l

In [21]:
for p in iv2_paths:
    files = os.listdir(os.path.join(ray_path, p))
    if 'error.txt' in files:
        continue
    
    print(p, files)

PPO_InvManagement-v2_ea14d_00005_5_lr=1e-06,vf_loss_coeff=0.5_2020-11-05_09-10-11 ['params.json', 'events.out.tfevents.1604556611.ubuntu', 'result.json', 'params.pkl', 'progress.csv']
PPO_InvManagement-v2_ea14d_00000_0_lr=1e-05,vf_loss_coeff=1.0_2020-11-04_12-55-48 ['events.out.tfevents.1604483748.ubuntu', 'params.json', 'result.json', 'params.pkl', 'progress.csv']
PPO_InvManagement-v2_ea14d_00002_2_lr=1e-06,vf_loss_coeff=1.0_2020-11-04_23-12-13 ['events.out.tfevents.1604520733.ubuntu', 'params.json', 'result.json', 'params.pkl', 'progress.csv']


In [23]:
def load_policy(path, checkpoint_id="500", pickle=True):
    if pickle:
        with open(f"{path}/params.pkl", "rb") as read_file:
            config = pickle.load(read_file)
    else:
        with open(f"{path}/params.json", "r") as read_file:
            config = json.load(read_file)
    split_path = path.split('/')
    algo = [j for i, j in enumerate(split_path) if split_path[i-1] == 'ray_results'][0].lower()
    if hasattr(agents, algo):
        agent = getattr(agents, algo)
    
    trainer = getattr(agent, algo.upper() + 'Trainer')(config, env=create_env(config['env']))
    trainer.restore(f"{path}/checkpoint_{checkpoint_id}/checkpoint-{checkpoint_id}")
    return trainer

In [25]:
?tune.run